import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [16]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, first
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when
from pyspark.sql.functions import monotonically_increasing_id 

create local SparkSession

In [2]:
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('Trainee') \
                    .getOrCreate()

read csv with inferschema

In [9]:
df = spark.read \
          .option('header', True) \
          .csv('./ds_salaries.csv')

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [22]:
df = spark.read \
            .option('inferschema',True) \
            .csv('./ds_salaries.csv')

write schema of scv on screen

In [10]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: string (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: string (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [2]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('work_year', StringType(), True),
    StructField('experiecne_level', StringType(), True),
    StructField('employment_type', StringType(), True),
    StructField('job_title', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('salary_currency', StringType(), True),
    StructField('salary_in_usd', IntegerType(), True),
    StructField('employee_residence', StringType(), True),
    StructField('remote_ratio', IntegerType(), True),
    StructField('company_location', StringType(), True),
    StructField('company_size', StringType(), True),
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [3]:
spark = SparkSession.builder.master('local[*]').appName('Trainee').getOrCreate()
df = spark.read \
          .option('header', True) \
          .format('csv') \
          .schema(schema) \
          .load('./ds_salaries.csv')

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [4]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experiecne_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [10]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experiecne_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [11]:
df.toPandas()

,id,work_year,experiecne_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [5]:
df_job_title = df.dropDuplicates(['job_title'])

print all rows from df_job_titles without truncating jobs

In [6]:
df_job_title.show(truncate=False)

+---+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|id |work_year|experiecne_level|employment_type|job_title                         |salary |salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|77 |2021     |MI              |PT             |3D Computer Vision Researcher     |400000 |INR            |5409         |IN                |50          |IN              |M           |
|52 |2020     |EN              |FT             |AI Scientist                      |300000 |DKK            |45896        |DK                |50          |DK              |S           |
|344|2022     |EX              |FT             |Analytics Engineer              

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [7]:
df_analytic = df_job_title.groupBy('job_title') \
                          .agg(max('salary_in_usd').alias('max_salary'),
                               min('salary_in_usd').alias('min_salary'),
                               avg('salary_in_usd').alias('avg_salary'))

print all rows from df_analytic without trancating jobs

In [8]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+----------+----------+
|job_title                               |max_salary|min_salary|avg_salary|
+----------------------------------------+----------+----------+----------+
|3D Computer Vision Researcher           |5409      |5409      |5409.0    |
|Lead Data Engineer                      |125000    |125000    |125000.0  |
|Head of Machine Learning                |79039     |79039     |79039.0   |
|Data Specialist                         |165000    |165000    |165000.0  |
|Data Analytics Lead                     |405000    |405000    |405000.0  |
|Machine Learning Scientist              |260000    |260000    |260000.0  |
|Lead Data Analyst                       |87000     |87000     |87000.0   |
|Data Engineering Manager                |59303     |59303     |59303.0   |
|Staff Data Scientist                    |105000    |105000    |105000.0  |
|ETL Developer                           |54957     |54957     |54957.0   |
|Director of

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [9]:
windowSpec = Window.orderBy(col('avg_salary').desc())
df_analytic = df_analytic.withColumn('id', row_number().over(windowSpec))

print all data from df_analytic

In [10]:
df_analytic.toPandas()

,job_title,max_salary,min_salary,avg_salary,id
0,Financial Data Analyst,450000,450000,450000.0,1
1,Data Analytics Lead,405000,405000,405000.0,2
2,Director of Data Science,325000,325000,325000.0,3
3,Machine Learning Scientist,260000,260000,260000.0,4
4,Head of Data,235000,235000,235000.0,5
5,Principal Data Engineer,200000,200000,200000.0,6
6,Data Science Manager,190200,190200,190200.0,7
7,Lead Data Scientist,190000,190000,190000.0,8
8,Analytics Engineer,175000,175000,175000.0,9
9,Principal Data Analyst,170000,170000,170000.0,10


it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [11]:
df_analytic = df_analytic.select(['id', 'job_title', 'avg_salary', 'min_salary', 'max_salary'])

print df_analytic now

In [12]:
df_analytic.show()

+---+--------------------+----------+----------+----------+
| id|           job_title|avg_salary|min_salary|max_salary|
+---+--------------------+----------+----------+----------+
|  1|Financial Data An...|  450000.0|    450000|    450000|
|  2| Data Analytics Lead|  405000.0|    405000|    405000|
|  3|Director of Data ...|  325000.0|    325000|    325000|
|  4|Machine Learning ...|  260000.0|    260000|    260000|
|  5|        Head of Data|  235000.0|    235000|    235000|
|  6|Principal Data En...|  200000.0|    200000|    200000|
|  7|Data Science Manager|  190200.0|    190200|    190200|
|  8| Lead Data Scientist|  190000.0|    190000|    190000|
|  9|  Analytics Engineer|  175000.0|    175000|    175000|
| 10|Principal Data An...|  170000.0|    170000|    170000|
| 11|     Data Specialist|  165000.0|    165000|    165000|
| 12|Machine Learning ...|  150000.0|    150000|    150000|
| 13|      Data Architect|  150000.0|    150000|    150000|
| 14|Principal Data Sc...|  148261.0|   

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [28]:
df_exp_lvl = df.groupBy('experiecne_level') \
                .agg(first('work_year').alias('work_year'),
                     first('experiecne_level').alias('experiecne_level'),
                     first('employment_type').alias('employment_type'),
                     first('job_title').alias('job_title'),
                     first('salary').alias('salary'),
                     first('salary_currency').alias('salary_currency'),
                     first('employee_residence').alias('employee_residence'),
                     first('remote_ratio').alias('remote_ratio'),
                     first('company_location').alias('company_location'),
                     first('company_size').alias('company_size'),
                     max('salary_in_usd').alias('max_salary'))

print here df_exp_lvl

In [29]:
df_exp_lvl.show()

+----------------+---------+----------------+---------------+--------------------+------+---------------+------------------+------------+----------------+------------+----------+
|experiecne_level|work_year|experiecne_level|employment_type|           job_title|salary|salary_currency|employee_residence|remote_ratio|company_location|company_size|max_salary|
+----------------+---------+----------------+---------------+--------------------+------+---------------+------------------+------------+----------------+------------+----------+
|              EN|     2020|              EN|             FT|        Data Analyst| 72000|            USD|                US|         100|              US|           L|    250000|
|              EX|     2020|              EX|             FT|Director of Data ...|325000|            USD|                US|         100|              US|           L|    600000|
|              MI|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [45]:
df_join = df.join(df_exp_lvl, on='experiecne_level', how='inner')
df_best = df.filter(col('salary_in_usd')== col('max_salary'))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `max_salary` cannot be resolved. Did you mean one of the following? [`salary`, `work_year`, `id`, `job_title`, `company_size`].;
'Filter (salary_in_usd#7 = 'max_salary)
+- Relation [id#0,work_year#1,experiecne_level#2,employment_type#3,job_title#4,salary#5,salary_currency#6,salary_in_usd#7,employee_residence#8,remote_ratio#9,company_location#10,company_size#11] csv


print df_best

drop duplicates if exist by experience_level

print df_best

create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

print df_new_best

write df_new_best like 1.csv and load then it to df_final

print df_final

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

print df_final

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system